In [39]:
# from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
lang = "py"
run_lang = "python"

In [ ]:
# import wandb

# api = wandb.Api()
# entity, project = "loubnabnl", "scaling_laws"  # set to your entity and project 
# runs = api.runs(entity + "/" + project) 

# summary_list, config_list, name_list = [], [], []
# for run in runs: 
#     # .summary contains the output keys/values for metrics like accuracy.
#     #  We call ._json_dict to omit large files 
#     summary_list.append(run.summary._json_dict)

#     # .config contains the hyperparameters.
#     #  We remove special values that start with _.
#     config_list.append(
#         {k: v for k,v in run.config.items()
#          if not k.startswith('_')})

#     # .name is the human-readable name of the run.
#     name_list.append(run.name)

# orig_runs_df = pd.DataFrame({
#     "summary": summary_list,
#     "config": config_list,
#     "name": name_list
#     })

# def expand_dict_to_columns(df, col):
#     return pd.concat([df.drop([col], axis=1),  pd.json_normalize(df[col])], axis=1)

# run_df = expand_dict_to_columns(orig_runs_df, "summary")
# run_df = expand_dict_to_columns(run_df, "config")

# keep_cols = [c for c in run_df if "lm-loss-validation/TEST_" not in c]
# run_df = run_df[keep_cols]

# df_run = run_df[run_df["_step"] >= .99 * run_df["train_iters"]]
# df_run.head(2)

In [40]:
DEFAULT_PROJECT_DIR = "/fsx-onellm/margaretli/env_srcs/xlf/xlformers_n/scaling/data"

col_names = ['C', 'D', 'N', 'lr', 'Avg Train Loss', 'Max Train Loss', 'C4 Eval PPL', 'Wiki Eval PPL', 'C4 Eval Loss', 'Wiki Eval Loss']

def read_data(csv_file, loss_name='C4 Eval Loss', col_names=col_names):
    mins_only = []
    df = pd.read_csv(csv_file, usecols=col_names,)
    df.dropna(subset=[loss_name], inplace=True)

    df = df.loc[(df['lr'] >= 0)]
    n_vals = df['N'].unique()
    d_vals = sorted(df['D'].unique())
    for n in n_vals:
        for d in d_vals:
            cd_df = df[(df['N'] == n) & (df['D'] == d)]
            if cd_df.empty:
                continue
            min_index = cd_df[loss_name].idxmin()
            # print(min_index)
            # print(cd_df)
            # print(cd_df.loc[min_index])
            mins_only.append(cd_df.loc[min_index])

    mins_only_df = pd.DataFrame(mins_only)
    # print(mins_only_df.N.values[:5].astype(np.longdouble))
    # print(mins_only_df['N'].values[:5].astype(np.longdouble))
    # print(np.fromstring(mins_only_df.N.values[:5], dtype=np.longdouble))

    print(mins_only_df)
    # df.rename(columns={})
    return mins_only_df

In [53]:
import torch
import numpy as np

def loss(inp, params):
    a, b, e, alpha, beta = params[0], params[1], params[2], params[3], params[4]
    pre_lse = torch.stack([a - alpha*torch.log(inp[:, 0]), b - beta*torch.log(inp[:, 1]), e.expand((inp.shape[0]))])
    post_lse = torch.logsumexp(pre_lse, dim=0)
    # huber_loss = torch.nn.functional.huber_loss(post_lse, torch.log(inp[:, 2]), delta=1e-3, reduction='none')
    huber_loss = torch.nn.functional.huber_loss(post_lse, torch.log(inp[:, 2]), delta=10, reduction='none')
    return huber_loss.sum()

def minimize_loss(inp, init_params=[6, 6, -1, 0.28, 0.32], steps=50):
    params = torch.nn.Parameter(data=torch.Tensor(init_params))
    
    lbfgs = torch.optim.LBFGS([params],
                    lr=1e-1,
                    history_size=10, 
                    max_iter=20, 
                    line_search_fn="strong_wolfe")

    def closure():
        lbfgs.zero_grad()
        l = loss(inp, params)
        l.backward()
        return l

    history_lbfgs = []
    for i in range(steps):
        l = lbfgs.step(closure)
    return l, params

In [54]:
loss_name = "C4 Eval Loss"
csv_file = "/fsx-onellm/margaretli/env_srcs/xlf/xlformers_n/scaling/data/data.csv"
project_dir=DEFAULT_PROJECT_DIR

fit_df = read_data(csv_file=csv_file, loss_name=loss_name)
# fit_df = fit_df[fit_df['N'] < 50000000]

inp = torch.Tensor([[N, D, L] for N, D, L in 
                    zip(fit_df["N"], fit_df["D"], fit_df[loss_name])])
inp.require_grad = True

min_loss = 1e10
for a in tqdm(np.linspace(0, 10, 5)):
    for b in np.linspace(0, 10, 5):
        for e in np.linspace(-1, 2, 4):
            for alpha in np.linspace(0, 1, 4):
                for beta in np.linspace(0, 1, 4):
                    l, params = minimize_loss(inp, [a, b, e, alpha, beta])
                    if l < min_loss:
                        min_loss = l
                        best_params = params.detach().numpy()

print(min_loss)
print(best_params)

        lr  Avg Train Loss  Max Train Loss  C4 Eval PPL  Wiki Eval PPL  \
4    0.004           4.957           5.185      146.194        216.300   
12   0.004           4.717           5.044      119.259        166.031   
17   0.004           4.453           4.659       89.148        115.380   
21   0.004           3.999           4.218       61.665         70.939   
25   0.004           3.736           4.058       46.984         47.592   
29   0.004           3.533           3.764       39.858         35.852   
33   0.002           4.925           5.171      141.632        224.183   
37   0.002           4.683           5.024      114.945        168.517   
42   0.004           3.880           4.109       55.653         62.826   
46   0.004           3.603           3.968       41.767         39.030   
50   0.004           3.448           3.680       36.775         32.393   
54   0.004           3.384           3.679       34.108         29.042   
58   0.004           3.312           3

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [04:05<00:00, 49.09s/it]

tensor(0.0083, grad_fn=<SumBackward0>)
[ 1.6370947e+00  1.2307936e+01 -4.4104099e+01  3.7521522e-02
  5.9921348e-01]


In [57]:
opt_alpha = best_params[-2]
opt_beta = best_params[-1]

opt_a =  opt_beta / (opt_alpha+opt_beta)
opt_b =  opt_alpha / (opt_alpha+opt_beta)

A = np.exp(best_params[0])
B = np.exp(best_params[1])
G = ((opt_alpha*A)/(opt_beta*B))**(1/(opt_alpha+opt_beta))

print("a", opt_a)
print("b", opt_b)
print("G", G)

def opt_N_D(C):
    opt_N = G*(C/6)**opt_a
    opt_D = (1/G)*(C/6)**opt_b
    return opt_N, opt_D

a 0.941072
b 0.058928
G 6.791991589901064e-10


In [58]:
scaling = []

for C in [1.92e19, 1.21e20, 1.23e22, 5.76e23, 3.85e24, 9.90e24, 3.43e25, 1.27e26, 1.30e28]:
    N, D = opt_N_D(C)
    scaling.append(
        {"compute": f"{C:e}",
         "parameters (B)": f"{N/1e9:.2f}",
         "tokens (B)": f"{D/1e9:.2f}",
        }
    )
     
pd.DataFrame(scaling)

,compute,parameters (B),tokens (B)
0,1.920000e+19,0.18,18.13
1,1.210000e+20,1.00,20.21
2,1.230000e+22,77.25,26.54
3,5.760000e+23,2883.82,33.29
4,3.850000e+24,17234.12,37.23
5,9.900000e+24,41917.26,39.36
6,3.430000e+25,134974.20,42.35
7,1.270000e+26,462657.10,45.75
8,1.300000e+28,36053325.81,60.10


In [47]:
def scaling_law(N, D, params):
    a, b, e, alpha, beta = params
    A = np.exp(a)
    B = np.exp(b)
    E = np.exp(e)
    
    L = E + (A / (N**alpha)) + (B /(D**beta))
    
    return L
scaling_law(1e10, 100e9, best_params)

2.2020082572609923

In [60]:
import plotly.express as px
import plotly.graph_objects as go

color_map={
           "3e+18": "orange",
           "6e+18": "black",
           "3e+18": "brown",
           "1e+19": "green",
           "3e+19": "purple",
           "6e+19": "red",
           "1e+20": "blue",
           "3e+20": "pink",
           "6e+20": "gold",
           "1e+21": "silver",
          }
fit_df.head(4)

# fit_df['C'] = fit_df['C'].astype(str)
fig = px.scatter(fit_df, x='N', y=loss_name, color='C', 
                 log_x=True, color_discrete_map=color_map)

for compute in color_map.keys():
    tmp_df = fit_df[fit_df['C'] == compute]
    df_d = list() 
    for _, row in tmp_df.iterrows():
        pred = scaling_law(row['N'], row['D'], best_params)
        df_d.append({'prediction': pred, 'N': row['N'], 'D': row['D']})
    fig2 = px.line(pd.DataFrame(df_d).sort_values('N'), 
                   x='N', y='prediction', log_x=True)
    fig2.update_traces(line_color=color_map[compute], line_width=2)
    fig = go.Figure(data=fig.data + fig2.data)

fig.show()

KeyError: 'N'